In [ ]:
using LinearAlgebra
using Random

In [ ]:
function combsum(n,r)
    sum = 0
    for i in 0:r
        sum = sum + binomial(n,i)
    end
    sum
end

function genmatrix(n,r)
    if (r==0)
        g = ones(1,2^n)
        return g
    elseif (r==n)
        g = I(2^n)
        return g
    else
        k = combsum(n-1,r-1)
        g = hcat(vcat(genmatrix(n-1,r),zeros(k,2^(n-1))),vcat(genmatrix(n-1,r),genmatrix(n-1,r-1)))
        return g
    end
end

In [ ]:
function energy(weight_size, codeword)
    sum = 0
    for i in codeword
        sum+=i
    end
    return abs(weight_size-sum)
end

In [ ]:
function minweight_codeword(n,r,coefficient)
    flag = 0
    A = Array{Int}(undef,((n-r),n))
    while flag==0
        A = rand([0,1],(n-r),n)
        if rank(A) == n-r
           flag=1 
        end
    end
    #println(A)
    x = coefficient*A
    #println(x)
    x = x.%2
    b = rand([0,1],n)
    #println(b)
    c_incidence = zeros(2^n)
    positions = zeros(2^(n-r))
    for i in 1:n
        for j in 1:2^(n-r)
            x[j,i] = x[j,i]⊻b[i]
            positions[j] = positions[j] + (x[j,i])*(2^(n-i))
        end
    end
    #print(positions)
    for i in positions
        c_incidence[2^n - Int(i)] = 1
    end
    #println(c_incidence)
    return c_incidence
end

In [ ]:
function metropolis_sampler(fixed_codeword, beta, weight_size, M, n, r, coefficient)
    c_prev = deepcopy(fixed_codeword)
    for i in 1:M
        c_incidence = minweight_codeword(n,r,coefficient)
        #println(energy(0,c_incidence))
        c_next = zeros(2^n)
        for j in 1:2^n 
            c_next[j] = c_incidence[j] + c_prev[j]
        end
        c_next = c_next.%2
        shift_threshold = exp(beta*(energy(weight_size,c_prev) - energy(weight_size, c_next)))
        shift = rand()
        if shift<=shift_threshold
            c_prev=deepcopy(c_next)
        end
    end
    return c_prev
end

In [ ]:
M = 10000000
M2 = 1
n = 9
r = 4
g = genmatrix(n,r)
k = combsum(n,r)

In [ ]:
coefficient = Array{Int, 2}(undef,(n-r),2^(n-r)) #Binary representations are stored column wise, leftmost digit is now topmost digit, then transposed
for i in 0:(2^(n-r)-1)
    temp = string(i, base=2, pad = n-r)
    for j in (1:(n-r))
        coefficient[j,i+1] = parse(Int,temp[j])
    end
end
coefficient = transpose(coefficient)

In [ ]:
versioninfo()

In [ ]:
Threads.nthreads()

In [ ]:
weights = Array{Float64}(undef,129)
count = 1
for weight_size in [20]#128:256#loop to range over different weights; range(0,2^(n-2) + 1)
    weight_size = 4*weight_size
    previous = big(0.0)
    #println(weight_size)
    estimate = big(2.0)^k
    beta = 0
    delta = 0.001
    samples_final = []
    while abs(estimate - previous) > delta 
        Y = 0
        samples = zeros(M2)
        for j in 1:M2
            sum = 0
            random_message = rand([0,1], k)
            fixed_codeword = transpose(random_message)*g
            fixed_codeword = fixed_codeword.%2
            sampled_codeword =  metropolis_sampler(fixed_codeword, beta, weight_size, M, n, r, coefficient)
            #println(sampled_codeword)
            #println(10000)
            for p in sampled_codeword
                sum = sum + p
            end
            samples[j] = sum
            Y = Y + exp((-1)*(energy(weight_size,sampled_codeword))*(1/n))
        end
        #println(Y)
        previous = estimate
        estimate = (estimate*Y)/M2
        if estimate<10^10
            estimate = 0
            break
        end
        beta += 1/n
        #println(estimate, samples)
        if (estimate - previous) <delta
            samples_final = samples
        end
    end
    println(weight_size, samples_final, estimate)
    # weights[count] = estimate
    # println(weights)
    # count = count + 1
end